# TCR co-occurence clustering

In [1]:
# I run the clustering on the gpu, maybe an environment with these dependencies will take some time to set up. Feel free to ask help.

import cugraph
import cudf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import faiss
import anndata

/media/pablo/Windows_files/00-master/02 - Semester2/[R1] Research_internship_1/.venv/lib/python3.10/site-packages/cudf/utils/gpu_utils.py:89: UserWarning: A GPU with NVIDIA Volta™ (Compute Capability 7.0) or newer architecture is required.
Detected GPU 0: NVIDIA GeForce GTX 1050                                                                                                                                                                                                                                         
Detected Compute Capability: 6.1
  warnings.warn(


In [2]:
cugraph.__version__

'24.02.00'

Hyperparameters

In [3]:
K_NN = 100

Read in count matrix (AnnData object)

In [4]:
counts = anndata.read_h5ad("data/counts_absence_presence_only.h5ad")

In [5]:
counts.obs

,Dataset,subject_id,Virus Diseases,Age,Biological Sex,Racial Group,Tissue Source,repertoire_size
sample_name,,,,,,,,
860011283_TCRB,COVID-19-HUniv12Oct,748180,COVID-19 Positive,86 Years,Female,Caucasian,Blood,129743.0
INCOV053-AC-3_TCRB,COVID-19-ISB,053,COVID-19 Positive,86 Years,Male,Caucasian,Blood,86436.0
KH20-09670_TCRB,COVID-19-DLS,550040039,COVID-19 Positive,84 Years,Male,Asian or Pacific Islander,Blood,45859.0
BS-EQ-23-T1-replacement_TCRB,COVID-19-NIH/NIAID,0000142,COVID-19 Positive,49 Years,Male,Caucasian,"Blood,gDNA",57115.0
BS-HS-157_TCRB,COVID-19-NIH/NIAID,NaN,NaN,NaN,NaN,NaN,"Blood,gDNA",23186.0
...,...,...,...,...,...,...,...,...
BS-GIGI_36-replacement_TCRB,COVID-19-NIH/NIAID,0000471,COVID-19 Positive,70 Years,Female,Caucasian,"Blood,gDNA",52262.0
1328-CM-933_TCRB,COVID-19-NIH/NIAID,NaN,NaN,NaN,NaN,NaN,"Buffy Coat,gDNA",106779.0
1566746BW_TCRB,COVID-19-BWNW,1566746,COVID-19 Positive,21 Years,Female,Native American or Alaska Native,Blood,330053.0


In [6]:
gpu_resource = faiss.StandardGpuResources()

modify X to change distance metric used

In [7]:
X = counts.X.T.todense()
X = np.ascontiguousarray(X).astype(np.float32)

# X = X/X.sum(axis=0)  # scale the features: fraction instead of counts. Need to experiment with this, does this have an impact? (doesnt really seem to improve things)
# X = normalize(X, norm="l1", axis=0) # test: alternative way of scaling features through L2
# X = X-X.mean(axis=0) # Center the columns to scale features?

X = X-X.mean(axis=1)[:,None]  # Center the rows. Use for adjusted cosine = approx pearson correlation. 
faiss.normalize_L2(X) # L2 normalize counts so that they sum up to one so that cosine similarity is measured (even though we use a dot product index)

Alternative method: include close neighbors:

In [8]:
# def gaussian(d, sigma):
    # return np.exp(-d**2 / (2 * sigma**2))

# X = np.ascontiguousarray(counts.layers["tcrdist_closest"].T.todense()).astype(np.float32)
# X = np.apply_along_axis(gaussian, 1, X, sigma=6) # distance to similarity
# X = X-X.mean(axis=1)[:,None]  # Center the rows. Use for adjusted cosine = approx pearson correlation. 
# faiss.normalize_L2(X) # L2 normalize counts so that they sum up to one so that cosine similarity is measured (even though we use a dot product index)

## K-nearest neighbor graph construction

In [9]:
# construct index
idx = faiss.GpuIndexFlatIP(gpu_resource, X.shape[1]) # Inner product index
idx.add(X)

Find the k-NNs for each TCR:

In [10]:
%%time
D,I = idx.search(X, K_NN) # search. We need to figure out which value of k works best.

CPU times: user 1min 13s, sys: 141 ms, total: 1min 13s
Wall time: 1min 14s


In [11]:
# save result matrices (intermediate step)

np.save("data/D", D)
np.save("data/I", I)

create k-NN graph:

In [12]:
D, I = np.load("data/D.npy"), np.load("data/I.npy") # load result matrices if needed

In [ ]:
# D and I matrix to edgelist

edgelist = set()
recursive_edgelist = []

for s,t,sim in zip(
    np.repeat(np.arange(I.shape[0]), K_NN).astype(int), #source
    I[:,:K_NN].ravel(),      #target
    D[:,:K_NN].ravel()):     #distance
        t = int(t)
        if s<t:
            edgelist.add((s,t))
        if t<s:
            if (t,s) in edgelist:
                recursive_edgelist.append((s,t,sim))

del edgelist

: 

In [ ]:
source, target, similarity = zip(*recursive_edgelist)

edge_df = cudf.DataFrame({
    "source":source,
    "target":target,
    "weight":similarity

})

edge_df.source = edge_df.source.astype(np.int32)
edge_df.target = edge_df.target.astype(np.int32)

: 

In [ ]:
G = cugraph.from_cudf_edgelist(edge_df, source="source", destination="target", edge_attr="weight") # create weighted graph object
# G = cugraph.from_cudf_edgelist(edge_df, source="source", destination="target") # create unweighted graph object

: 

In [ ]:
G.is_weighted()

: 

In [ ]:
G.number_of_nodes()

: 

In [ ]:
G.number_of_edges()

: 

In [ ]:
cugraph.connected_components(G)["labels"].nunique()

: 

## Clustering

In [ ]:
scl = cugraph.spectralModularityMaximizationClustering(G, num_clusters=1000, num_eigen_vects=50)
# scl = cugraph.spectralBalancedCutClustering(G, num_clusters=500, num_eigen_vects=50, kmean_tolerance=1E-8, kmean_max_iter=1000)

: 

In [ ]:
scl = scl.to_pandas()

: 

In [ ]:
sizes = scl["cluster"].value_counts().to_numpy()

: 

In [ ]:
fig, ax = plt.subplots(figsize=(5,3),dpi=200)
sns.histplot(sizes)
sns.despine()
ax.set_ylabel("Number of modules")
ax.set_xlabel("Module size")

: 

In [ ]:
counts.var["cluster"] = scl.to_pandas().assign(index=scl["vertex"].astype(str)).set_index("index")["cluster"]

: 

In [ ]:
counts.var["cluster"] = counts.var["cluster"].fillna(-1).astype(int)

: 

In [ ]:
counts.var.to_csv("results/spectral_clustering.csv")
edge_df.to_csv("results/edgelist.csv")

: 